<h3>Importing Data Cleaning and Merging it.</h3>

In [52]:
import pandas as pd

# Load the data
people_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\01_people.csv")
abilities_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\02_abilities.csv")
education_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\03_education.csv")
skills_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\06_skills.csv")
experience_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\04_experience.csv")
person_skill_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\05_person_skills.csv")



<h3>Abilities CSV</h3>
<p>The Data is Structured like multiple repeated rows with skills of individual People. People are identified by Person_id(primary Key)
we will be keeping only first 5000 people for our project the dataset has like 50000+ people with thier data in it</p>

In [32]:
abilities_aggregated = abilities_df.groupby('person_id')['ability'].apply(lambda x: ', '.join(x)).reset_index()

# Save the aggregated data to a new CSV
abilities_aggregated = abilities_aggregated.head(4999)
abilities_aggregated.to_csv('02_abilities.csv', index=False)

<h3>People CSV</h3>
<p>we will be removing the unnecsary columns from this csv which are email, linkdein and phone which are at average 90% null and we dont need them in our project</p>

In [ ]:
columns_to_drop = ['email', 'phone', 'linkedin']
df_cleaned = people_df.drop(columns=columns_to_drop)

# Save the cleaned data to a new CSV file
df_cleaned = df_cleaned.head(5000)
df_cleaned.to_csv('01_people.csv', index=False)


print("Columns removed and cleaned file saved as '01_people_cleaned.csv'")

Columns removed and cleaned file saved as '01_people_cleaned.csv'


<h3>Education CSV</h3>
<p>There are 5 columns person_id, institution, program, start_date and location, we will be removing start_date and Location since we dont need them. we will also be joining multiple person_id's rows.</p>

In [34]:

# Merge the education data into one row per person_id, combining all institutions and programs
df_education_merged = education_df.groupby('person_id').agg({
    'institution': lambda x: ', '.join(x.dropna().unique()),  # Concatenate unique institutions
    'program': lambda x: ', '.join(x.dropna().unique()),      # Concatenate unique programs
}).reset_index()

df_education_merged = df_education_merged.head(5000)
# Save the cleaned and merged data to a new CSV file
df_education_merged.to_csv('03_education.csv', index=False)


print("Education data has been merged and cleaned.")


Education data has been merged and cleaned.


<h3>Experience CSV</h3>
<p>So this one needs more work than any other. we will first merge multiple person_id rows into 1, than we will calculate the years a person worked for, if missing we will add zero instead. </p>

In [ ]:
import pandas as pd
from datetime import datetime

# Ensure that 'title' and 'firm' columns are treated as strings
experience_df['title'] = experience_df['title'].fillna('').astype(str)
experience_df['firm'] = experience_df['firm'].fillna('').astype(str)

# Convert start_date and end_date to datetime, coercing any errors
experience_df['start_date'] = pd.to_datetime(experience_df['start_date'], errors='coerce')
experience_df['end_date'] = pd.to_datetime(experience_df['end_date'], errors='coerce')

# Fill missing end_date values with the current date
experience_df['end_date'].fillna(datetime.now(), inplace=True)

# Calculate the duration in years (difference between start and end date)
experience_df['duration_years'] = (experience_df['end_date'] - experience_df['start_date']).dt.days / 365.25

# Group by 'person_id' and aggregate data
df_experience_grouped = experience_df.groupby('person_id').agg({
    'title': ' '.join,  # Concatenate job titles for each person
    'firm': ' '.join,   # Concatenate firm names for each person
    'start_date': 'min',  # Keep earliest start date
    'end_date': 'max',    # Keep latest end date
    'duration_years': 'sum'  # Sum the durations for all jobs
}).reset_index()

# Save the cleaned and grouped experience data to a new CSV file
df_experience_grouped = df_experience_grouped.head(5000)
df_experience_grouped.to_csv('04_experience_grouped.csv', index=False)



In [ ]:
import pandas as pd
from datetime import datetime

# Load your 04_experience dataset
df = pd.read_csv('C:/Users/DELL/Desktop/Projects/DataScience/AI-powered-Resume-Builder-and-Reviewer/data/Preprocessing/04_experience_cleaned.csv')

# Fill missing end dates with today's date for entries with 0.0 duration
current_date = datetime.today().strftime('%Y-%m-%d')

# Replace empty or None End Dates with today's date
df['end_date'].fillna(current_date, inplace=True)

# Ensure the columns 'Start Date' and 'End Date' are in datetime format (removing time part)
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date

# Recalculate the duration for those entries that have missing or invalid end dates
df['Calculated Duration'] = (pd.to_datetime(df['end_date']) - pd.to_datetime(df['start_date'])).dt.days / 365

# Optionally, update the duration for rows with original 0.0 duration values
df.loc[df['Calculated Duration'] == 0.0, 'Calculated Duration'] = (pd.to_datetime(df['end_date']) - pd.to_datetime(df['start_date'])).dt.days / 365

# Handle NaN or infinite values in the 'Calculated Duration' column
df['Calculated Duration'].fillna(0, inplace=True)  # Replace NaN with 0 (or another default value)
df['Calculated Duration'] = df['Calculated Duration'].replace([float('inf'), -float('inf')], 0)  # Replace infinite values with 0

# Round the 'Calculated Duration' to the nearest whole number (years) and convert to integer
df['Calculated Duration'] = df['Calculated Duration'].round().astype(int)

# Drop the 'Start Date', 'End Date', and any other columns related to time
df.drop(columns=['start_date', 'end_date', 'duration_years'], inplace=True)

# Save the updated DataFrame back to a CSV file if necessary
df.to_csv('C:/Users/DELL/Desktop/Projects/DataScience/AI-powered-Resume-Builder-and-Reviewer/data/Preprocessing/04_experience.csv', index=False)


<h3>Person Skills CSV</h3>
<p>So this one needs more work than any other. we will first merge multiple person_id rows into 1, than we will calculate the years a person worked for, if missing we will add zero instead. </p>